# How can I add a legend to a folium map?

In the discussion of these issues, ***How can I add a legend to a folium map?*** [@ColinTalbert](https://github.com/python-visualization/folium/issues/528#issuecomment-421445303) kicked it off. I found the solution he proposed very interesting. The solution is here: [*How does one add a legend (categorical) to a folium map*](https://nbviewer.jupyter.org/gist/talbertc-usgs/18f8901fc98f109f2b71156cf3ac81cd), adapted from [Tilemill Project](https://tilemill-project.github.io/tilemill/docs/guides/advanced-legends/)

[@araichev](https://github.com/python-visualization/folium/issues/528#issuecomment-624366588) adjusted some parameters, allowing the iteration of a given category, a fact that helps **a lot** the automation that I needed!!

Starting from that last code, I made some modifications that make it possible:
- Insert colors automatically from a color palette (using *seaborn*);
- I divided the function of [@araichev](https://github.com/mrcagney/examples_folium) into two, one of which modifies only the ***header*** of the *HTML map* (inserting the .js and script to make it draggable), while the other modifies the ***body*** (inserting the legend). I did it because it was the way I found to insert multiple subtitles!
- I adjusted some formatting to make it possible to move the legend again.
- Cleaned HMTL to insert.

In the ***colors.ipynb*** file I tested some color palettes from the seaborn. There are several others...

The legend must be created (for as many *layers* as you want), using:

```m = add_CategoricalLegend (m, 'Municipios', color_by_label = color_polygon)```

Change the HTML header only once, using:

```m = modify_HeaderLegend (m)```

# *Imports*

In [ ]:
import os
import folium
import pandas as pd
import branca as bc
import seaborn as sns
import geopandas as gpd
from folium.plugins import BeautifyIcon

# *Function*

In [ ]:
def modify_HeaderLegend(map_folium):    
    """
    """
    # Header to Add
    head = """
    {% macro header(this, kwargs) %}
    <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
    <script>$( function() {
        $( ".maplegend" ).draggable({
            start: function (event, ui) {
                $(this).css({
                    right: "auto",
                    top: "auto",
                    bottom: "auto"
                });
            }
        });
    });
    </script>
    {% endmacro %}
    """
    
    # Add Header
    macro = bc.element.MacroElement()
    macro._template = bc.element.Template(head)
    map_folium.get_root().add_child(macro)
    
    
    # CSS to Add (on Header)
    css = """
    {% macro header(this, kwargs) %}
    <style type='text/css'>
      .maplegend {
        position: absolute;
        z-index:9999;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        right: 10px;
        bottom: 20px;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 0px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    {% endmacro %}
    """
    
    # Add CSS (on Header)
    macro = bc.element.MacroElement()
    macro._template = bc.element.Template(css)
    map_folium.get_root().add_child(macro)    

    return map_folium

In [ ]:
def add_CategoricalLegend(map_folium, title, color_by_label):
    """
    """
    body = f"""
    <div id='maplegend {title}' class='maplegend'>
        <div class='legend-title'>{title}</div>
        <div class='legend-scale'>
            <ul class='legend-labels'>"""
    
    # Loop Categories
    for label, color in color_by_label.items():
        body += f"""
                <li><span style='background:{color}'></span>{label}</li>"""
    
    body += """
            </ul>
        </div>
    </div>
    """   

    # Add Body
    body = bc.element.Element(body)
    map_folium.get_root().html.add_child(body)

    return map_folium

# Add Legend to Map

## Polygons

In [ ]:
# Load geodata features
path = os.path.join('data', 'Santos.geojson')
gdf = gpd.read_file(path).to_crs('epsg:4326')

gdf.head(3)

In [ ]:
# Column with category
categories = 'Nome_Municipio'

# Set palette
palette_polygon = 'colorblind'

# Get list of unique values
categories = set(gdf[categories])
categories = list(categories)
categories.sort()
print(categories)

# See the palette chosed
pal = sns.color_palette(palette_polygon, n_colors=len(categories))
sns.palplot(pal)
colors = pal.as_hex()

# If I want set colors mannualy 
#myColors = ['#9b59b6', '#3498db', '#95a5a6', '#e74c3c', '#34495e', '#2ecc71']
#colors = myColors

# Set dictionary
color_polygon = dict(zip(categories, colors))

In [ ]:
# Initialize Map
m = folium.Map(tiles=None)
folium.TileLayer(tiles='cartodbpositron', attr='', name='Basemap').add_to(m)

# Add GeoJSON
blocks = folium.GeoJson(gdf,
                        name='Municípios',
                        style_function=lambda x: {
                            'fillColor': color_polygon[x['properties']['Nome_Municipio']],
                            'color': color_polygon[x['properties']['Nome_Municipio']],
                            'weight': 1,
                            'fillOpacity': 0.3},
                       ).add_to(m)

# Extend
m.fit_bounds(m.get_bounds())

# Add Map Legend
m = modify_HeaderLegend(m)
m = add_CategoricalLegend(m , 'Municipios', color_by_label=color_polygon)

# Map
m.save(os.path.join('maps', 'map.html'))
m

## Lines

In [ ]:
# Load geodata features
path = os.path.join('data', 'trajetos.geojson')
gdf = gpd.read_file(path).to_crs('epsg:4326')
gdf.head(3)

In [ ]:
# Column with category
categories = 'trajetos'

# Set palette
palette_line = 'dark'

# Get list of unique values
categories = set(gdf[categories])
categories = list(categories)
categories.sort()
print(categories)

# See the palette chosed
pal = sns.color_palette(palette_line, n_colors=len(categories))
sns.palplot(pal)
colors = pal.as_hex()

# If I want set colors mannualy 
#myColors = ['#9b59b6', '#3498db']
#colors = myColors

# Set dictionary
color_line = dict(zip(categories, colors))

In [ ]:
# Initialize Map
m = folium.Map(tiles=None)
folium.TileLayer(tiles='cartodbpositron', attr='', name='Basemap').add_to(m)

# Add GeoJSON
traj = folium.GeoJson(gdf,
                      name='Trajetos',
                      style_function=lambda x: {
                          'color': color_line[x['properties']['trajetos']],
                          'weight': 4,
                      },
                     ).add_to(m)

# Extend
m.fit_bounds(m.get_bounds())

# Add Map Legend
m = modify_HeaderLegend(m)
m = add_CategoricalLegend(m, 'Trajetos', color_by_label=color_line)

# Map
m.save(os.path.join('maps', 'map.html'))
m

## Points

In [ ]:
# Load geodata features
path = os.path.join('data', 'EmpresasSantos.geojson')
gdf = gpd.read_file(path).to_crs('epsg:4326')
gdf.head(3)

In [ ]:
# Column with category
categories = 'neighborhood'

# Set palette
palette_point = 'Set1'

# Get list of unique values
categories = set(gdf[categories])
categories = list(categories)
categories.sort()
print(categories)

# See the palette chosed
pal = sns.color_palette(palette_point, n_colors=len(categories))
sns.palplot(pal)
colors = pal.as_hex()

# If I want set colors mannualy 
#myColors = ['#9b59b6', '#3498db', '#95a5a6', '#e74c3c', '#34495e', '#2ecc71']
#colors = myColors

# Set dictionary
color_points = dict(zip(categories, colors))

In [ ]:
# Initialize Map
m = folium.Map(tiles=None)
folium.TileLayer(tiles='cartodbpositron', attr='', name='Basemap').add_to(m)

# Add Lat Long
gdf['Lat'] = gdf['geometry'].y
gdf['Long'] = gdf['geometry'].x

# Markers
layer = folium.FeatureGroup(name='Points')
for index, row in gdf.iterrows():
    if row['neighborhood'] in color_points.keys():
        # Set Style (could improve)
        style = ('color:' + color_points[row['neighborhood']] +';font-size:10px;')
                
        # Add Markers
        folium.Marker(location=[row['latitude'], row['longitude']],
                      icon=BeautifyIcon(icon='star',
                                        inner_icon_style=style,
                                        background_color='transparent',
                                        border_color='transparent',
                                       )
                     ).add_to(layer)
layer.add_to(m)

# Extend
m.fit_bounds(m.get_bounds())

# Add Map Legend
m = modify_HeaderLegend(m)
m = add_CategoricalLegend(m, 'Empresas', color_by_label=color_points)

# Map
m

# Multiple Legends

In [ ]:
# Initialize Map
m = folium.Map(tiles=None)
folium.TileLayer(tiles='cartodbpositron', attr='', name='Basemap').add_to(m)

# ------------------------------------------------------------------------------
# Load geodata features
url = 'https://raw.githubusercontent.com/michelmetran/package_folium/master/data/Santos.geojson'
gdf = gpd.read_file(url).to_crs('epsg:4326')

# Column with category
categories = 'Nome_Municipio'

# Set palette
palette_polygon = 'colorblind'

# Get list of unique values
categories = set(gdf[categories])
categories = list(categories)
categories.sort()

# See the palette chosed
pal = sns.color_palette(palette_polygon, n_colors=len(categories))
colors = pal.as_hex()

# Set dictionary
color_polygon = dict(zip(categories, colors))

# Initialize Map
m = folium.Map(tiles=None)
folium.TileLayer(tiles='cartodbpositron', attr='', name='Basemap').add_to(m)

# Add GeoJSON
blocks = folium.GeoJson(url,
                        name='Municipios',
                        style_function=lambda x: {
                            'fillColor': color_polygon[x['properties']['Nome_Municipio']],
                            'color': color_polygon[x['properties']['Nome_Municipio']],
                            'weight': 1,
                            'fillOpacity': 0.3},
                        embed=False,
                       ).add_to(m)

# Add Map Legend
m = add_CategoricalLegend(m, 'Municipio', color_by_label=color_polygon)

# ------------------------------------------------------------------------------
# LINES

# Load geodata features
url = 'https://raw.githubusercontent.com/michelmetran/package_folium/master/data/trajetos.geojson'
gdf = gpd.read_file(url).to_crs('epsg:4326')

# Column with category
categories = 'trajetos'

# Set palette
palette_line = 'dark'

# Get list of unique values
categories = set(gdf[categories])
categories = list(categories)
categories.sort()

# See the palette chosed
pal = sns.color_palette(palette_line, n_colors=len(categories))
colors = pal.as_hex()

# Set dictionary
color_line = dict(zip(categories, colors))

# Add GeoJSON
traj = folium.GeoJson(url,
                      name='Trajetos',
                      style_function=lambda x: {
                          'color': color_line[x['properties']['trajetos']],
                          'weight': 4,
                      },
                      embed=False,
                     ).add_to(m)

# Add Map Legend
m = add_CategoricalLegend(m, 'Trajetos', color_by_label=color_line)

# ------------------------------------------------------------------------------
# POINTS

# Load geodata features
url = 'https://raw.githubusercontent.com/michelmetran/package_folium/master/data/EmpresasSantos.geojson'
gdf = gpd.read_file(path).to_crs('epsg:4326')

# Column with category
categories = 'neighborhood'

# Set palette
palette_point = 'Set1'

# Get list of unique values
categories = set(gdf[categories])
categories = list(categories)
categories.sort()

# See the palette chosed
pal = sns.color_palette(palette_point, n_colors=len(categories))
colors = pal.as_hex()

# Set dictionary
color_points = dict(zip(categories, colors))

# Add Lat Long
gdf['Lat'] = gdf['geometry'].y
gdf['Long'] = gdf['geometry'].x

# Markers
layer = folium.FeatureGroup(name='Points')
for index, row in gdf.iterrows():
    if row['neighborhood'] in color_points.keys():
        # Set Style (could improve)
        style = ('color:' + color_points[row['neighborhood']] +';font-size:10px;')
                
        # Add Markers
        folium.Marker(location=[row['latitude'], row['longitude']],
                      icon=BeautifyIcon(icon='star',
                                        inner_icon_style=style,
                                        background_color='transparent',
                                        border_color='transparent',
                                       ),
                      embed=False,
                     ).add_to(layer)
layer.add_to(m)


# Add Map Legend
m = add_CategoricalLegend(m, 'Empresas', color_by_label=color_points)

# Extend
m.fit_bounds(m.get_bounds())
folium.LayerControl().add_to(m)

# Add only once
m = modify_HeaderLegend(m)

# Map
m.save(os.path.join('maps', 'map.html'))
m